In [ ]:
import os
import sys
import pandas as pd
import numpy as np

import metpy.calc as mpcalc
import metpy.plots as mplots
from metpy.units import units

import matplotlib.pyplot as plt
import matplotlib
from mpl_toolkits.axes_grid1.inset_locator import inset_axes

from scipy.interpolate import interp1d

from PIL import Image

#makes all plots visible within the Jupyter notebook
%matplotlib inline


In [ ]:
def plot_skewTs(file_date, plot_hodograph = True):
    """ Make skew-T/hodograph figures for a given day's dropsonde profiles
    
    PARAMETERS
    ----------
    file_date : the day (YYYYMMDD, string format) for which you want to plot dropsonde profile skew-Ts
    plot_hodograph : determines whether or not to plot an inset hodograph (True/False)
    
    RETURNS
    ----------
    fig : matplotlib skew-T figures for each dropsonde in the given day's "final_dropsonde_YYYYMMDD.csv" file
    
    """
    
    day_folder = os.path.join(os.getcwd(), file_date)
    drop_final_name = os.path.join(day_folder, 'final_dropsonde_' + file_date + '.csv')
    
    drop_csv = pd.read_csv(drop_final_name)
    drop_times = drop_csv['Time [UTC]'].unique()  #sorted() = goes through files in alphabetical order
    
    #initialize some plot visualizations
    
    #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
    #matplotlib.rcParams['font.size'] = 15
    matplotlib.rcParams['font.family'] = 'arial'
    matplotlib.rcParams['axes.labelsize'] = 18
    matplotlib.rcParams['axes.titlesize'] = 20
    #matplotlib.rcParams['axes.titlesize'] = 26
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    matplotlib.rcParams['axes.titleweight'] = 'bold'
    matplotlib.rcParams['xtick.labelsize'] = 18
    matplotlib.rcParams['ytick.labelsize'] = 18
    matplotlib.rcParams['legend.fontsize'] = 18
    #matplotlib.rcParams['legend.facecolor'] = 'w'
    #matplotlib.rcParams['axes.facecolor'] = 'w'
    matplotlib.rcParams['hatch.linewidth'] = 0.3

    for time in drop_times:
        #print (time)    #for debugging purposes
        
        df0 = drop_csv[drop_csv['Time [UTC]'] == time].copy()
        df = df0.iloc[::-1]  #reverses the dataframe (row-based) to go from surface to upper-level

        #add appropriate units to the pressure, temperature, dewpoint, and wind data
        pres = df['Pressure [mb]'].values * units.hPa   #hPa = mb
        #filtered_pres = pres[::2]      #only every other pressure value for quicker profile/skew-T creation
        temp = df['Temperature [C]'].values * units.degC
        #filtered_temp = temp[::2]      ##only every other temp. value for quicker skew-T creation
        dwpt = df['Dew Point [C]'].values * units.degC
        #wnd_spd = df['Wind Speed [m/s]'].values * units('m/s')
        wnd_spd = (df['Wind Speed [m/s]'].values * units('m/s')).to(units.knots)  #convert wind speed to knots
        wnd_dir = df['Wind Direction [deg]'].values * units.deg
        #hght = df['Height [m]'].values * units.meter

        #calculate the parcel path/profile at the near-surface for the given environment
        
        #profile = mpcalc.parcel_profile(pres, temp[0], dwpt[0])  #returns temps in Kelvin
        try:
            profile = mpcalc.parcel_profile(pres, temp[0], dwpt[0])  #returns temps in Kelvin
        except:
            print (f'Could not plot {time} skew-T, likely because this dropsonde contains no moisture data or pressure increases between at least two points in the sounding. Using scipy.signal.medfilt may fix the latter.')
            continue
            
        profile = profile.to('degC')

        #calculate the LCL and wind components
        lcl = mpcalc.lcl(pres[0], temp[0], dwpt[0])
        wind_comps = mpcalc.wind_components(wnd_spd, wnd_dir)  #returns u,v values in whatever unit wnd_spd is in
        u = wind_comps[0]
        v = wind_comps[1]

        #initialize the figure
        fig = plt.figure(figsize = (12,12)) 
        save_name = os.path.join(day_folder, 'skewt_' + time[11:13] + time[14:16] + time[17:19] + '.png')

        #Initialize the skew-T figure/subplot
        skew = mplots.SkewT(fig)

        #Plot the data for the skew-T
        skew.plot(pres, temp, 'darkorange', linewidth = 2)
        skew.plot(pres, dwpt, 'cornflowerblue', linewidth = 2)
        #skew.plot(lcl[0], lcl[1], 'yellow', marker = '*', markeredgecolor = 'k', markersize = 14)  #plot the LCL as a yellow star
        #skew.plot(filtered_pres, profile, 'k', linewidth = 2)
        skew.plot(pres, profile, 'k', linewidth = 2)
        skew.plot_barbs(pres[::60], u[::60], v[::60])
        #skew.shade_cape(filtered_pres, filtered_temp, profile)
        #skew.shade_cin(filtered_pres, filtered_temp, profile, dwpt[::2])
        skew.shade_cape(pres, temp, profile, alpha = 0.2)
        skew.shade_cin(pres, temp, profile, alpha = 0.2)
        skew.plot_dry_adiabats(t0 = np.arange(-90, 321, 10) * units.degC, alpha = 0.3)   #range is large to cover whole plot for all possible profiles
        skew.plot_moist_adiabats(t0 = np.arange(-90, 81, 10) * units.degC, alpha = 0.3)  #range is large to cover whole plot for all possible profiles
        skew.ax.set_xlim(-40,40)
        skew.ax.set_ylim(1000,200)
        skew.ax.set_xlabel('Temperature [$\degree$C]')
        skew.ax.set_ylabel('Pressure [hPa]')
        
        #if just one sounding text file is inputted, then plot a hodograph in the upper right-hand corner of the figure
        if plot_hodograph:
            axh = inset_axes(skew.ax, '35%', '35%', loc = 'upper right')
            h = mplots.Hodograph(axh, component_range = 80.)
            h.add_grid(increment = 20)
            
            try:
                h.plot_colormapped(u, v, wnd_spd);  # Plot a line colored by wind speed
            except:
                fig.text(0.755, 0.643, 'No Wind Data', horizontalalignment='center', 
                         verticalalignment='center', fontsize = 20)
                
            skew.ax.set_title(f'{time} UTC Dropsonde Skew-T Diagram and Hodograph') #title created based on dropsonde time
            plt.tight_layout()
            #plt.show()
            plt.savefig(save_name)  #bbox_inches = 'tight' is not cooperating with the inset hodograph for some reason
            plt.close()
        else:
            skew.ax.set_title(f'{time} UTC Dropsonde Skew-T Diagram') #title created based on dropsonde time
            #skew.ax.set_title(f'{time[11:]} UTC Dropsonde Skew-T') #for CPEX-CV BAMS article, Figure 4
            #plt.show()
            plt.savefig(save_name, bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
            plt.close()
        
        #decrease file size of the image by 4x without noticeable image effects (if using Matplotlib)!
        #(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
        im = Image.open(save_name)
        try:
            im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
        except:
            im2 = im.convert('P')  #use this for older version of PIL/Pillow if the above line doesn't work, though this line will have isolated, extremely minor image effects due to only using 256 colors instead of the 3-part RGB scale
        im2.save(save_name)
        im.close()
        im2.close()


In [ ]:
def median_sonde(df_drop):
    """ Calculate the median dropsonde profile for a given list of dropsondes
    
    PARAMETERS
    ----------
    df_drop : a Pandas DataFrame of the list of dropsondes (originated from the Dropsonde Metrics CSV file)
    
    RETURNS
    ----------
    median_dropsonde : a Pandas DataFrame of the median dropsonde profile (including all dropsonde variables) for the given list of dropsondes
    
    """
    
    pnew = np.linspace(1000, 400, 61, endpoint = True) #new pressure array for interpolation with 10 hPa spacing; endpoint = True --> 400 is included as the last value
        
    for x in range(len(df_drop)):
        drop_date = str(df_drop['Date'].iloc[x])
        drop_time = str(df_drop['Time'].iloc[x]).zfill(6)
        sonde_datetime = drop_date[:4] + '-' + drop_date[4:6] + '-' + drop_date[6:] + ' ' + drop_time[:2] + ':' + drop_time[2:4] + ':' + drop_time[4:]

        #locate and read in the given dropsonde's data file
        day_folder = os.path.join(os.getcwd(), drop_date)   
        drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + drop_date + '.csv')
        drop_csv = pd.read_csv(drop_csv_path)
        drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
        drop_csv_use = drop_csv_use.iloc[::-1]   #reverses the dataframe (row-based) to go from surface to upper-level
        
        #interpolate the dropsonde's data to 10 hPa pressure spacing, ranging from 1000 - 400 hPa
            #all relevant CPEX/CPEX-AW sondes (isolated, organized, non-TC related, no In Precip) go down to 
                #at least 980 hPa and up to at least 408 hPa, so interpolating from 1000 - 400 hPa can be done without any drastic estimations
        
        temp_int_scheme = interp1d(drop_csv_use['Pressure [mb]'], drop_csv_use['Temperature [C]'], kind = 'cubic', bounds_error = False)  #cubic interpolation scheme, not linear since we're interpolating by pressure, not height
        temp_int = temp_int_scheme(pnew)
        
        dwpt_int_scheme = interp1d(drop_csv_use['Pressure [mb]'], drop_csv_use['Dew Point [C]'], kind = 'cubic', bounds_error = False)  #cubic interpolation scheme, not linear since we're interpolating by pressure, not height
        dwpt_int = dwpt_int_scheme(pnew)
        
        #store each dropsonde's interpolated data into dataframes (one for each variable)
        if x == 0:
            temp_profiles_df = pd.DataFrame(temp_int)
            dwpt_profiles_df = pd.DataFrame(dwpt_int)
        else:
            temp_profiles_df = pd.concat((temp_profiles_df, pd.Series(temp_int)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths (which should be none with the uniform interpolation) are filled in with NaNs
            dwpt_profiles_df = pd.concat((dwpt_profiles_df, pd.Series(dwpt_int)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths (which should be none with the uniform interpolation) are filled in with NaNs
            
    median_temp_profile = temp_profiles_df.median(axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    median_dwpt_profile = dwpt_profiles_df.median(axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    
    median_dropsonde = pd.DataFrame({'Pressure [mb]': pnew, 'Temperature [C]': median_temp_profile, 'Dew Point [C]': median_dwpt_profile})
    
    return median_dropsonde  #one median dropsonde profile (including all variables) for all profiles in df_drop
  
        

In [ ]:
def median_IQR_sonde(df_drop):
    """ Calculate the median dropsonde profile for a given list of dropsondes
    
    PARAMETERS
    ----------
    df_drop : a Pandas DataFrame of the list of dropsondes (originated from the Dropsonde Metrics CSV file)
    
    RETURNS
    ----------
    median_dropsonde : a Pandas DataFrame of the median dropsonde profile (including all dropsonde variables) for the given list of dropsondes
    first_IQR_dropsonde : a Pandas DataFrame of the 1st IQR dropsonde profile (including all dropsonde variables) for the given list of dropsondes
    third_IQR_dropsonde : a Pandas DataFrame of the 3rd IQR dropsonde profile (including all dropsonde variables) for the given list of dropsondes
    
    """
    
    pnew = np.linspace(1000, 400, 61, endpoint = True) #new pressure array for interpolation with 10 hPa spacing; endpoint = True --> 400 is included as the last value
        
    for x in range(len(df_drop)):
        drop_date = str(df_drop['Date'].iloc[x])
        drop_time = str(df_drop['Time'].iloc[x]).zfill(6)
        sonde_datetime = drop_date[:4] + '-' + drop_date[4:6] + '-' + drop_date[6:] + ' ' + drop_time[:2] + ':' + drop_time[2:4] + ':' + drop_time[4:]

        #locate and read in the given dropsonde's data file
        day_folder = os.path.join(os.getcwd(), drop_date)   
        drop_csv_path = os.path.join(day_folder, 'final_dropsonde_' + drop_date + '.csv')
        drop_csv = pd.read_csv(drop_csv_path)
        drop_csv_use = drop_csv[drop_csv['Time [UTC]'] == sonde_datetime].copy()
        drop_csv_use = drop_csv_use.iloc[::-1]   #reverses the dataframe (row-based) to go from surface to upper-level
        
        #interpolate the dropsonde's data to 10 hPa pressure spacing, ranging from 1000 - 400 hPa
            #all relevant CPEX/CPEX-AW sondes (isolated, organized, non-TC related, no In Precip) go down to 
                #at least 980 hPa and up to at least 408 hPa, so interpolating from 1000 - 400 hPa can be done without any drastic estimations
        
        temp_int_scheme = interp1d(drop_csv_use['Pressure [mb]'], drop_csv_use['Temperature [C]'], kind = 'cubic', bounds_error = False)  #cubic interpolation scheme, not linear since we're interpolating by pressure, not height
        temp_int = temp_int_scheme(pnew)
        
        dwpt_int_scheme = interp1d(drop_csv_use['Pressure [mb]'], drop_csv_use['Dew Point [C]'], kind = 'cubic', bounds_error = False)  #cubic interpolation scheme, not linear since we're interpolating by pressure, not height
        dwpt_int = dwpt_int_scheme(pnew)
        
        #store each dropsonde's interpolated data into dataframes (one for each variable)
        if x == 0:
            temp_profiles_df = pd.DataFrame(temp_int)
            dwpt_profiles_df = pd.DataFrame(dwpt_int)
        else:
            temp_profiles_df = pd.concat((temp_profiles_df, pd.Series(temp_int)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths (which should be none with the uniform interpolation) are filled in with NaNs
            dwpt_profiles_df = pd.concat((dwpt_profiles_df, pd.Series(dwpt_int)), axis = 1, ignore_index = True)  #add profile as a new column to the df; differences in profile lengths (which should be none with the uniform interpolation) are filled in with NaNs
            
    median_temp_profile = temp_profiles_df.median(axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    median_dwpt_profile = dwpt_profiles_df.median(axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    
    first_IQR_temp_profile = temp_profiles_df.quantile(q = 0.25, axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    first_IQR_dwpt_profile = dwpt_profiles_df.quantile(q = 0.25, axis = 1)     #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    
    third_IQR_temp_profile = temp_profiles_df.quantile(q = 0.75, axis = 1)  #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    third_IQR_dwpt_profile = dwpt_profiles_df.quantile(q = 0.75, axis = 1)     #axis of 1 = across all columns (so for each row); NaNs are ignored by default in Pandas
    
    median_dropsonde = pd.DataFrame({'Pressure [mb]': pnew, 'Temperature [C]': median_temp_profile, 'Dew Point [C]': median_dwpt_profile})
    first_IQR_dropsonde = pd.DataFrame({'Pressure [mb]': pnew, 'Temperature [C]': first_IQR_temp_profile, 'Dew Point [C]': first_IQR_dwpt_profile})
    third_IQR_dropsonde = pd.DataFrame({'Pressure [mb]': pnew, 'Temperature [C]': third_IQR_temp_profile, 'Dew Point [C]': third_IQR_dwpt_profile})
    
    return median_dropsonde, first_IQR_dropsonde, third_IQR_dropsonde  #one median, 1st IQR, and 3rd IQR dropsonde profile (including all variables) for all profiles in df_drop
  

In [ ]:
def plot_median_skewTs(drop_filepath, convective_type, only_inflow = False):
    """ Make median skew-T/hodograph figures for a given day's dropsonde profiles
    
    PARAMETERS
    ----------
    drop_filepath : the filepath of the Dropsonde Metrics CSV file
    convective_type : the convective type of the dropsondes for which to plot the median skew-T/hodograph 
                      (Isolated/Organized/Scattered --> case sensitive!)
    only_inflow : determines whether or not to plot median skew-Ts using only inflow dropsondes (True/False)
    
    RETURNS
    ----------
    fig : matplotlib median skew-T figures for Isolated/Organized/Scattered dropsondes in the drop_filepath file
    
    """
    
    #initialize some plot visualizations 
    
    #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
    #matplotlib.rcParams['font.size'] = 15
    matplotlib.rcParams['font.family'] = 'arial'
    matplotlib.rcParams['axes.labelsize'] = 18
    matplotlib.rcParams['axes.titlesize'] = 20
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    matplotlib.rcParams['axes.titleweight'] = 'bold'
    matplotlib.rcParams['xtick.labelsize'] = 18
    matplotlib.rcParams['ytick.labelsize'] = 18
    matplotlib.rcParams['legend.fontsize'] = 18
    #matplotlib.rcParams['legend.facecolor'] = 'w'
    #matplotlib.rcParams['axes.facecolor'] = 'w'
    matplotlib.rcParams['hatch.linewidth'] = 0.3
    
    TC_days = [20170619, 20170620, 20210826, 20210828, 20210901, 20210904,
              '20170619', '20170620', '20210826', '20210828', '20210901', '20210904']  #including TDs as TCs (from both CPEX and CPEX-AW)

    df_orig = pd.read_csv(drop_filepath)
    df_noTC = df_orig[~df_orig['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC
    
    df_use = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    if only_inflow:    
        df_use = df_use[(df_use['Low-level Inflow Sonde'] == 'Yes') | (df_use['Mid-level Inflow Sonde'] == 'Yes')].copy()                 #filter out non-inflow dropsondes
    else:
        pass

    df_convective_type = df_use[df_use['Primary Convective Type'] == convective_type].copy()
    
    df = median_sonde(df_convective_type)  #one median dropsonde profile (including all variables) for all profiles in df_convective_type

    #add appropriate units to the pressure, temperature, and dewpoint data
    pres = df['Pressure [mb]'].values * units.hPa   #hPa = mb
    #filtered_pres = pres[::2]      #only every other pressure value for quicker profile/skew-T creation
    temp = df['Temperature [C]'].values * units.degC
    #filtered_temp = temp[::2]      ##only every other temp. value for quicker skew-T creation
    dwpt = df['Dew Point [C]'].values * units.degC

    #calculate the parcel path/profile at the near-surface for the given environment
    profile = mpcalc.parcel_profile(pres, temp[0], dwpt[0])  #returns temps in Kelvin
    profile = profile.to('degC')

    #calculate the LCL
    lcl = mpcalc.lcl(pres[0], temp[0], dwpt[0])

    #initialize the figure
    fig = plt.figure(figsize = (12,12))       

    #Initialize the skew-T figure/subplot
    skew = mplots.SkewT(fig)

    #Plot the data for the skew-T
    skew.plot(pres, temp, 'darkorange', linewidth = 2)
    skew.plot(pres, dwpt, 'cornflowerblue', linewidth = 2)
    #skew.plot(lcl[0], lcl[1], 'yellow', marker = '*', markeredgecolor = 'k', markersize = 14)  #plot the LCL as a yellow star
    #skew.plot(filtered_pres, profile, 'k', linewidth = 2)
    skew.plot(pres, profile, 'k', linewidth = 2)
    #skew.shade_cape(filtered_pres, filtered_temp, profile)
    #skew.shade_cin(filtered_pres, filtered_temp, profile, dwpt[::2])
    skew.shade_cape(pres, temp, profile, alpha = 0.2)
    skew.shade_cin(pres, temp, profile, alpha = 0.2)
    skew.plot_dry_adiabats(t0 = np.arange(-90, 321, 10) * units.degC, alpha = 0.3)   #range is large to cover whole plot for all possible profiles
    skew.plot_moist_adiabats(t0 = np.arange(-90, 81, 10) * units.degC, alpha = 0.3)  #range is large to cover whole plot for all possible profiles
    skew.ax.set_xlim(-15,35)
    skew.ax.set_ylim(1000,350)
    skew.ax.set_xlabel('Temperature [$\degree$C]')
    skew.ax.set_ylabel('Pressure [hPa]')
    skew.ax.text(0.2125, 0.05, 'In Precip Dropsondes Excluded', transform = skew.ax.transAxes,
                 horizontalalignment = 'center', verticalalignment = 'center', fontsize = 18,
                 bbox = {'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
    
    if only_inflow:
        skew.ax.set_title(f'Median Skew-T Diagram for {convective_type} Inflow Dropsondes')
        save_name = os.path.join(os.path.join('/', 'Users', 'brodenkirch', 'Desktop'), convective_type + '_median_SkewT_inflow.png')
    else:
        skew.ax.set_title(f'Median Skew-T Diagram for {convective_type} Dropsondes')
        save_name = os.path.join(os.path.join('/', 'Users', 'brodenkirch', 'Desktop'), convective_type + '_median_SkewT.png')
        
    #plt.show()
    plt.savefig(save_name, bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
    plt.close()

    #decrease file size of the image by 4x without noticeable image effects (if using Matplotlib)!
    #(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
    im = Image.open(save_name)
    try:
        im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
    except:
        im2 = im.convert('P')  #use this for older version of PIL/Pillow if the above line doesn't work, though this line will have isolated, extremely minor image effects due to only using 256 colors instead of the 3-part RGB scale
    im2.save(save_name)
    im.close()
    im2.close()


In [ ]:
def plot_median_IQR_skewTs(drop_filepath, convective_type, only_inflow = False):
    """ Make median skew-T/hodograph figures for a given day's dropsonde profiles
    
    PARAMETERS
    ----------
    drop_filepath : the filepath of the Dropsonde Metrics CSV file
    convective_type : the convective type of the dropsondes for which to plot the median skew-T/hodograph 
                      (Isolated/Organized/Scattered --> case sensitive!)
    only_inflow : determines whether or not to plot median skew-Ts using only inflow dropsondes (True/False)
    
    RETURNS
    ----------
    fig : matplotlib median and IQR skew-T profiles figure for Isolated/Organized/Scattered dropsondes in the drop_filepath file
    
    """
    
    #initialize some plot visualizations 
    
    #matplotlib.rcParams['axes.facecolor'] = [0.9,0.9,0.9]
    #matplotlib.rcParams['font.size'] = 15
    matplotlib.rcParams['font.family'] = 'arial'
    matplotlib.rcParams['axes.labelsize'] = 18
    matplotlib.rcParams['axes.titlesize'] = 20
    matplotlib.rcParams['axes.labelweight'] = 'bold'
    matplotlib.rcParams['axes.titleweight'] = 'bold'
    matplotlib.rcParams['xtick.labelsize'] = 18
    matplotlib.rcParams['ytick.labelsize'] = 18
    matplotlib.rcParams['legend.fontsize'] = 18
    #matplotlib.rcParams['legend.facecolor'] = 'w'
    #matplotlib.rcParams['axes.facecolor'] = 'w'
    matplotlib.rcParams['hatch.linewidth'] = 0.3
    
    TC_days = [20170619, 20170620, 20210826, 20210828, 20210901, 20210904,
              '20170619', '20170620', '20210826', '20210828', '20210901', '20210904']  #including TDs as TCs (from both CPEX and CPEX-AW)

    df_orig = pd.read_csv(drop_filepath)
    df_noTC = df_orig[~df_orig['Date'].isin(TC_days)].copy()  #filter out cases associated with a TD/TC
    
    df_use = df_noTC[(df_noTC['Environment Falling In'] != 'In Precip') & (df_noTC['Environment Falling In'] != 'Clear Far')].copy()  #filter out In Precip and Clear Far dropsondes
    if only_inflow:    
        df_use = df_use[(df_use['Low-level Inflow Sonde'] == 'Yes') | (df_use['Mid-level Inflow Sonde'] == 'Yes')].copy()                 #filter out non-inflow dropsondes
    else:
        pass
    
    df_convective_type = df_use[df_use['Primary Convective Type'] == convective_type].copy()

    df_median, df_firstIQR, df_thirdIQR = median_IQR_sonde(df_convective_type)  #one median, first IQR, and third IQR dropsonde profile (including all variables) for all profiles in df_convective_type

    #add appropriate units to the pressure, temperature, and dewpoint data
    pres_median = df_median['Pressure [mb]'].values * units.hPa     #hPa = mb
    temp_median = df_median['Temperature [C]'].values * units.degC
    dwpt_median = df_median['Dew Point [C]'].values * units.degC

    pres_firstIQR = df_firstIQR['Pressure [mb]'].values * units.hPa     #hPa = mb
    temp_firstIQR = df_firstIQR['Temperature [C]'].values * units.degC
    dwpt_firstIQR = df_firstIQR['Dew Point [C]'].values * units.degC

    pres_thirdIQR = df_thirdIQR['Pressure [mb]'].values * units.hPa     #hPa = mb
    temp_thirdIQR = df_thirdIQR['Temperature [C]'].values * units.degC
    dwpt_thirdIQR = df_thirdIQR['Dew Point [C]'].values * units.degC

    #initialize the figure
    fig = plt.figure(figsize = (12,12))       

    #Initialize the skew-T figure/subplot
    skew = mplots.SkewT(fig)

    #Plot the data for the skew-T
    skew.plot(pres_median, temp_median, 'darkorange', linewidth = 2, linestyle = '-')
    skew.plot(pres_median, dwpt_median, 'cornflowerblue', linewidth = 2, linestyle = '-')
#     skew.plot(pres_firstIQR, temp_firstIQR, 'darkorange', linewidth = 2, linestyle = '--')
#     skew.plot(pres_firstIQR, dwpt_firstIQR, 'cornflowerblue', linewidth = 2, linestyle = '--')
#     skew.plot(pres_thirdIQR, temp_thirdIQR, 'darkorange', linewidth = 2, linestyle = '--')
#     skew.plot(pres_thirdIQR, dwpt_thirdIQR, 'cornflowerblue', linewidth = 2, linestyle = '--')
    
    #Fill in areas between the 1st and 3rd IQRs for both temperature and dewpoint
    skew.ax.fill_betweenx(pres_median, temp_firstIQR, temp_thirdIQR, color = 'darkorange', alpha = 0.3)
    skew.ax.fill_betweenx(pres_median, dwpt_firstIQR, dwpt_thirdIQR, color = 'cornflowerblue', alpha = 0.3)

    skew.plot_dry_adiabats(t0 = np.arange(-90, 321, 10) * units.degC, alpha = 0.3)   #range is large to cover whole plot for all possible profiles
    skew.plot_moist_adiabats(t0 = np.arange(-90, 81, 10) * units.degC, alpha = 0.3)  #range is large to cover whole plot for all possible profiles
    skew.ax.set_xlim(-15,35)
    skew.ax.set_ylim(1000,350)
    skew.ax.set_xlabel('Temperature [$\degree$C]')
    skew.ax.set_ylabel('Pressure [hPa]')
    skew.ax.text(0.2125, 0.05, 'In Precip Dropsondes Excluded', transform = skew.ax.transAxes,
                 horizontalalignment = 'center', verticalalignment = 'center', fontsize = 18,
                 bbox = {'facecolor': 'white', 'alpha': 0.5, 'pad': 10})
    
    if only_inflow:
        skew.ax.set_title(f'Median and IQR Skew-T Diagram for {convective_type} Inflow Dropsondes')
        save_name = os.path.join(os.path.join('/', 'Users', 'brodenkirch', 'Desktop'), convective_type + '_median_SkewT_inflow.png')
    else:
        skew.ax.set_title(f'Median and IQR Skew-T Diagram for {convective_type} Dropsondes')
        save_name = os.path.join(os.path.join('/', 'Users', 'brodenkirch', 'Desktop'), convective_type + '_median_SkewT.png')
        
    #plt.show()
    plt.savefig(save_name, bbox_inches = 'tight')  #bbox_inches = 'tight' will clip any additional white space around the image
    plt.close()

    #decrease file size of the image by 4x without noticeable image effects (if using Matplotlib)!
    #(good to use if you're producing a lot of images, see https://www.youtube.com/watch?v=fzhAseXp5B4)
    im = Image.open(save_name)
    try:
        im2 = im.convert('P', palette = Image.Palette.ADAPTIVE)
    except:
        im2 = im.convert('P')  #use this for older version of PIL/Pillow if the above line doesn't work, though this line will have isolated, extremely minor image effects due to only using 256 colors instead of the 3-part RGB scale
    im2.save(save_name)
    im.close()
    im2.close()


In [ ]:
plot_skewTs('20220922', plot_hodograph = False)  ##bbox_inches = 'tight' is not cooperating with the inset hodographs for some reason

#For Ben's specific purposes (you can ignore this cell):
#plot skew-T diagrams for each dropsonde for each CPEX/CPEX-AW/CPEX-CV convective case

# for x in sorted(os.listdir()):
#     if os.path.isdir(x) and x[0:4] == '2022':
#         if x != '20220916_prelim':
#             print (x)
#             plot_skewTs(x, plot_hodograph = False)


In [ ]:
# #to help Figures 3-4 in AGU paper (2023), plot median skew-T profiles for both Isolated and Organized convection
#     #make these median skew-T profile plots for all non-TC, non-Precip sondes and then for just inflow sondes

# drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
# #drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')

# #all sondes (non-TC, non-Precip)
# plot_median_skewTs(drop_filepath, convective_type = 'Isolated', only_inflow = False)
# plot_median_skewTs(drop_filepath, convective_type = 'Organized', only_inflow = False)

# #just inflow sondes (non-TC, non-Precip)
# plot_median_skewTs(drop_filepath, convective_type = 'Isolated', only_inflow = True)
# plot_median_skewTs(drop_filepath, convective_type = 'Organized', only_inflow = True)


In [ ]:
# #to help Figures 3-4 in AGU paper (2023), plot median and IQR skew-T profiles for both Isolated and Organized convection
#     #make these median skew-T profile plots for all non-TC, non-Precip sondes and then for just inflow sondes

# drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations.csv')
# #drop_filepath = os.path.join(os.getcwd(), 'Dropsonde_Metric_Calculations_CPEXCV.csv')

# #all sondes (non-TC, non-Precip)
# plot_median_IQR_skewTs(drop_filepath, convective_type = 'Isolated', only_inflow = False)
# plot_median_IQR_skewTs(drop_filepath, convective_type = 'Organized', only_inflow = False)

# #just inflow sondes (non-TC, non-Precip)
# plot_median_IQR_skewTs(drop_filepath, convective_type = 'Isolated', only_inflow = True)
# plot_median_IQR_skewTs(drop_filepath, convective_type = 'Organized', only_inflow = True)
